<a href="https://colab.research.google.com/github/dekanayake/25km/blob/main/google_places_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install turfpy

In [ ]:
import pandas as pd, numpy as np
import requests
import json
import time
final_data = []
from turfpy.measurement import destination, center
from geojson import Feature, FeatureCollection, Point
# Parameters

class ScanBox:
  topRight=[]
  topLeft=[]
  bottomRight=[]
  bottomLeft=[]

  def center(self):
    f1 = Feature(geometry=Point((self.topRight[0], self.topRight[1])))
    f2 = Feature(geometry=Point((self.topLeft[0], self.topLeft[1])))
    f3 = Feature(geometry=Point((self.bottomLeft[0], self.bottomLeft[1])))
    f3 = Feature(geometry=Point((self.bottomRight[0], self.bottomRight[1])))
    feature_collection = FeatureCollection([f1, f2, f3])
    return center(feature_collection).geometry.coordinates

  def __str__(self):
    return "topLeft:" + str(self.topLeft) +",topRight:" + str(self.topRight) + ",bottomLeft:" + str(self.bottomLeft) + ",bottomRight:" + str(self.bottomRight)

class AreaBox:
  boxes = []
  startingPoint=[]
  rows=0
  cols=0
  row=0
  column=0
  numberOfBoxes=0
  filledBoxes=1
  boxDistance=0

  def __init__(self, width, height, startingPoint, boxDistance):
    self.rows = int(width/boxDistance)
    self.cols = int(height/boxDistance)
    self.startingPoint = startingPoint
    self.numberOfBoxes = self.rows * self.cols
    self.boxDistance = boxDistance
    self.boxes = [[ScanBox() for j in range(self.cols)] for i in range(self.rows)]

  def __getBoxCellDataToFill(self):
    if self.column == 0 and self.row > 0:
      return (self.row, self.column,self.__getBox(self.row - 1,0).topRight)
    if self.row == 0 and self.column == 0:
      return (self.row, self.column,self.startingPoint)  
    return (self.row, self.column,self.__getBox(self.row,self.column - 1).bottomLeft)

  def _moveBoxIndex(self):
    if self.column == self.cols - 1:
      self.row += 1
      self.column = 0
    else:  
      self.column += 1 
      

  def addBox(self):
    while self.filledBoxes <= self.numberOfBoxes:
      row,column, startingPoint = self.__getBoxCellDataToFill()
      self.__updateBox(self.__getBox(row,column),startingPoint)
      self._moveBoxIndex()
      self.filledBoxes += 1



  def __getBox(self, row , column):
    columns = self.boxes[row]
    return columns[column]

  def __updateBox(self, scanBox, startingPoint):
    scanBox.topLeft = startingPoint
    origin = Feature(geometry=Point(scanBox.topLeft))
    distance = self.boxDistance
    bearing = 90
    options = {'units': 'km'}
    scanBox.topRight = destination(origin,distance,bearing,options).geometry.coordinates

    origin = Feature(geometry=Point(scanBox.topLeft))
    bearing = 180
    scanBox.bottomLeft = destination(origin,distance,bearing,options).geometry.coordinates

    origin = Feature(geometry=Point(scanBox.topRight))
    bearing = 180
    scanBox.bottomRight = destination(origin,distance,bearing,options).geometry.coordinates




origin = Feature(geometry=Point([145.282,-37.8862]))
distance = 90
bearing = 315
options = {'units': 'km'}
startingPoint = destination(origin,distance,bearing,options).geometry.coordinates
areaBox = AreaBox(180,180,startingPoint,10)
areaBox.addBox()

coordinates = []

boxes = pd.DataFrame(areaBox.boxes)
for row in range(0, areaBox.rows):
  for column in range(0, areaBox.cols):
     box = boxes.at[row,column]
     coordinates.append(str([box.center()[1],box.center()[0]]).strip('[]'))

keywords = ['park']
radius = '10000'
api_key = '[key]' #insert your Places API
for coordinate in coordinates:
  for keyword in keywords:
    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='+coordinate+'&radius='+str(radius)+'&keyword='+str(keyword)+'&key='+str(api_key)
    while True:
      print(url)
      respon = requests.get(url)
      jj = json.loads(respon.text)
      results = jj['results']
      for result in results:
        name = result['name']
        place_id = result ['place_id']
        lat = result['geometry']['location']['lat']
        lng = result['geometry']['location']['lng']
        rating = result['rating']
        types = result['types']
        vicinity = result['vicinity']
        data = [name, place_id, lat, lng, rating, types, vicinity]
        final_data.append(data)
      time.sleep(5)
      if 'next_page_token' not in jj:
        break
      else:
        next_page_token = jj['next_page_token']
        url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?key='+str(api_key)+'&pagetoken='+str(next_page_token)
labels = ['Place Name','Place ID', 'Latitude', 'Longitude', 'Rating','Types', 'Vicinity']
export_dataframe_1_medium = pd.DataFrame.from_records(final_data, columns=labels)
export_dataframe_1_medium.to_csv('export_dataframe_2_medium.csv')









# New Section